<center>
    <h1 style="width: 90%">Understanding Relationships Between Professor Info and Course Enrollment Rates in the Computer Science Department at the University of Maryland, College Park</h1>
    <h2 style="width: 90%">By Wesley Smith, Franco Edah, and Ohsun Kwon</h2>
</center>

When given the ability of choice, people nearly always seeks out the best possible selection to ensure their own success. So, it is not surprising that students at the University of Maryland, College Park commonly compare course ratings and grade data, amongst many other factors, in their choice of which courses and course sections to select each semester. In this project, our group seeks to better understand the relationships between professor rating and professor average GPA with how students seek to enroll in courses within the Department of Computer Science at the UMCP.

# Fetching Our Data

Before we are able to begin our analysis, we first need to retrieve the publicly available data about professors and courses within the Computer Science Department. Using APIs developed by our peer students at the University hosted on `umd.io` and `planetterp.com`, we are able to secure the necessary data to conduct our analyses.

Fetching large amounts of data from the `umd.io` and `planetterp.com` APIs consistently as we were working on our project turned out to be costly both in terms of the amount of time we were waiting for data to process, and costly for the APIs themselves, with us sometimes being rate-limited when fixing small mistakes. For reference, being rate-limited is when too many requests are sent to a public API, and the API limits requests from your address in an effort to conserve resources for others. 

In order to solve the costliness of fetching and re-fetching the data from these publicly available APIs, we instead fetch all of the data about CMSC (Dept. of Computer Science) courses at UMCP at the beginning of our project, store them in an SQLite database, and use this database to do necessary queries throughout our data analysis.

We chose the SQLite database over other initial solutions like MongoDB due to SQLite having native support in the Pandas library, along with the fact that storing our data in a table-like fashion made the most sense considering Pandas needs to represent it as a table as well. If we were to use MongoDB for example, we would need to query the database and then flatten our data to be placed into a Pandas dataframe for analysis anyways. By representing the data as a table from the beginning, we remove this rather redundant step when importing our data. Furthermore, keeping the data local in our own repository was also another plus, as needing to connect to a remote service for hosting our data could involve extra costs and complexity in our analysis.

Below is the code that scrapes the `umd.io` and `planetterp.com` APIs and stores the necessary data that we recieve in our local SQLite database.

## Connect to the Database

In [15]:
import sqlite3
conn = sqlite3.connect("project.db") # connect to the database
conn

## Fetch and Write Basic Course Info

First, we need to get a list of all of the courses that are available within the `umd.io` API for us to fetch. However, we cannot get all of our data in one API request because:
1. The API returns results in a paginated form, and we can only request up to 100 entries on each page. So, if there are more courses than 100 for a single set of request parameters, we need to keep fetching the next page until there are no more results to return.
2. The API is supposed to support being able to request all courses less than or equal to (`leq`) than a semester, but the API seems to be broken and does not accept strings longer than six characters. To provide the `leq` option, we would need to provide `202008|leq` for the semester parameter, which is 10 characters long. To work around this issue, we start at the current year (`2022`) and iteratively make a call for both the spring `01` semester and `08` fall semesters for each year, decrementing the year each iteration. 

In [12]:
import requests

def get_courses(page, semester):
    params = {"dept_id": "CMSC", "per_page": 100, "page": page, "semester": semester}
    return requests.get("https://api.umd.io/v1/courses", params=params).json()

def fetch_for_semester(semester):
    result = []
    page = 1
    while True:
        response = get_courses(page, semester)
        if "error_code" in response:
            if response["message"] == "We don't have data for this semester!":
                # theres no data to fetch! stop now
                break
            raise Exception(f"unknown error response: {response}")
                
        if len(response) == 0:
            # if we got no result, then we reached the last page
            break
            
        result += response
        page += 1
    return result

def fetch_for_year(year):
    result = []
    for semester_id in ["01", "08"]:
        result += fetch_for_semester(f"{year}{semester_id}")
    return result

courses = []
year = 2022
while True:
    result = fetch_for_year(year)
    # if we got no result, then we have reached the last year umd.io has data for
    if len(result) == 0:
        break
        
    courses += result
    year -= 1

len_courses = len(courses)
print(f"amount of courses fetched: {len_courses}")

amount of courses fetched: 880


Now, we will write this information to our database to be able to use later without re-fetching from the `umd.io` API. When we see duplicate courses, that means we have already placed these in our database and will ignore the error result.

As we are writing data to the database, we are parsing out the semester field of the API result to split it into the year and semester category that it represents. In the database, we represent the fall and spring semesters as the enum values `FALL` and `SPRING` respectively.

In [22]:
def parse_semester(semester_raw):
    semester_year = semester_raw[0:4]
    semester_id = int(semester_raw[4:])
    semester_enum = "FALL" if semester_id == 1 else "SPRING" if semester_id == 8 else None
    return semester_year, semester_enum

cur = conn.cursor()
cur.execute("DROP TABLE courses") # uncomment if you'd like to start over
cur.execute("""
CREATE TABLE IF NOT EXISTS courses (
    id CHAR(7) NOT NULL PRIMARY KEY,
    dept CHAR(4) NOT NULL,
    number VARCHAR(4) NOT NULL,
    year INT(4) NOT NULL,
    semester VARCHAR(10) NOT NULL,
    credits INT(1) NOT NULL
)
""")

for course in courses:
    course_id = course["course_id"]
    dept = course["dept_id"]
    number = course_id[len(dept):]
    year, semester = parse_semester(course["semester"])
    credits = int(course["credits"])
    cur.execute(
        "INSERT OR IGNORE INTO courses (id, dept, number, year, semester, credits) VALUES (?, ?, ?, ?, ?, ?)",
        (course_id, dept, number, year, semester, credits)
    )

print(list(cur.execute("SELECT * FROM courses LIMIT 5")))

[('CMSC100', 'CMSC', '100', 2022, 'FALL', 1), ('CMSC106', 'CMSC', '106', 2022, 'FALL', 4), ('CMSC122', 'CMSC', '122', 2022, 'FALL', 3), ('CMSC125', 'CMSC', '125', 2022, 'FALL', 3), ('CMSC131', 'CMSC', '131', 2022, 'FALL', 4)]


In [40]:
import requests
import sqlite3
import os
from operator import itemgetter
import math

# a map of grade value to the grade point value
GRADE_POINTS = {
    "A+": 4.0, "A": 4.0, "A-": 3.7, 
    "B+": 3.3, "B": 3.0, "B-": 2.7,
    "C+": 2.3, "C": 2.0, "C-": 1.7,
    "D+": 1.3, "D": 1.0, "D-": 0.5,
    "F": 0.0
}
SECTIONS_PER_PAGE = 100

# using the provided connection, ensure that all of our tables exist
def db_ensure_tables_exist(conn):
    # courses stores the overall info for a course
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS courses (
        id CHAR(7) NOT NULL PRIMARY KEY,
        dept CHAR(4) NOT NULL,
        number INT(4) NOT NULL,
        credits INT(1) NOT NULL
    )
    """)
    # course_professors stores the professors for a course
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS course_professors (
        course CHAR(7) NOT NULL,
        name VARCHAR(100) NOT NULL,
        CONSTRAINT primary_key PRIMARY KEY (course, name),
        FOREIGN KEY (course) REFERENCES courses(id)
    )
    """)
    # You cannot create a primary key containing a TEXT field, 
    # so we don't have one for this table.
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS professor_reviews (
        course CHAR(7) NOT NULL,
        prof_name VARCHAR(100) NOT NULL,
        rating INT(1) NOT NULL,
        review TEXT NOT NULL,
        FOREIGN KEY (course) REFERENCES courses(id),
        FOREIGN KEY (prof_name) REFERENCES course_professors(name)
    )
    """)
    # course_grades stores the gpa and number of drops for a course
    # in a given year and semester. the semester column is either
    # "FALL" or "SPRING".
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS course_grades (
        course CHAR(7) NOT NULL,
        prof_name VARCHAR(100),
        year INT(4) NOT NULL,
        semester VARCHAR(10) NOT NULL,
        gpa DOUBLE(2,2) NOT NULL,
        num_drops INT(5) NOT NULL,
        CONSTRAINT primary_key PRIMARY KEY (course, prof_name, year, semester),
        FOREIGN KEY (course) REFERENCES courses(id),
        FOREIGN KEY (prof_name) REFERENCES course_professors(name)
    )
    """)
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS course_section_seats (
        course CHAR(7) NOT NULL,
        section_number CHAR(4) NOT NULL,
        year INT(4) NOT NULL,
        semester VARCHAR(10) NOT NULL,
        seats_open INT(4) NOT NULL,
        seats_taken INT(4) NOT NULL,
        waitlist_size INT(4) NOT NULL,
        CONSTRAINT primary_key PRIMARY KEY (course, section_number, year, semester),
        FOREIGN KEY (course) REFERENCES courses(id)
    )
    """)

# write the information about a course to the provided connection
def db_write_course_info(conn, plt_terp_info):
    dept, number, credits, prof_names, reviews = itemgetter(
        "department",
        "course_number", 
        "credits",
        "professors",
        "reviews"
    )(plt_terp_info)
    cur = conn.cursor()

    id = f"{dept}{number}"
    cur.execute(
        "INSERT INTO courses (id, dept, number, credits) VALUES (?, ?, ?, ?)", 
        (id, dept, number, credits)
    )
    
    for prof_name in prof_names:
        cur.execute(
            "INSERT OR IGNORE INTO course_professors (course, name) VALUES (?, ?)",
            (id, prof_name)
        )
    
    for review in reviews:
        # perhaps do sentiment analysis here on the review text?
        cur.execute(
            "INSERT INTO professor_reviews (course, prof_name, rating, review) VALUES (?, ?, ?, ?)",
            (id, review["professor"], review["rating"], review["review"])
        )

# write the information about a courses grades to the provided connection
def db_write_course_grades(conn, course_id, plt_terp_grades):
    cur = conn.cursor()
    for entry in plt_terp_grades:
        num_grade_w = int(entry["W"])
        prof_name = entry["professor"]
        semester_raw = entry["semester"]
        year, semester = parse_semester(semester_raw)
            
        # if we can't identify fall or spring semester,
        # then the course isnt relevant to our data
        if semester == None:
            continue
            
        # loop over keys and values and check if key is a grade
        # name. if so, add it to our grade point sum and total
        # amount of grades.
        grade_point_sum = 0
        total_grades = 0
        for key, value in entry.items():
            if key not in GRADE_POINTS:
                continue
            
            grade_points = GRADE_POINTS[key]
            amt = int(value)
            total_grades += amt
            grade_point_sum += amt * grade_points
        
        gpa = grade_point_sum / total_grades
        cur.execute(
            """
            INSERT INTO course_grades (course, prof_name, year, semester, gpa, num_drops)
            VALUES (?, ?, ?, ?, ?, ?)
            ON CONFLICT(course, prof_name, year, semester) 
                DO UPDATE SET gpa = (gpa + excluded.gpa) / 2 AND num_drops = num_drops + excluded.num_drops
            """,
            (course_id, prof_name, year, semester, gpa, num_grade_w)
        )

def db_write_section_seats(conn, sections):
    cur = conn.cursor()
    for section in sections:
        course_id = section["course"]
        number = section["number"]
        year, semester = parse_semester(str(section["semester"]))
        seats_open = int(section["open_seats"])
        seats = int(section["seats"])
        seats_taken = seats - seats_open
        waitlist = int(section["waitlist"])
        cur.execute(
            """
            INSERT INTO course_section_seats (course, section_number, year, semester, seats_open, seats_taken, waitlist_size)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
            (course_id, number, year, semester, seats_open, seats_taken, waitlist)
        )

def get_all_sections(course_id, semester, num_sections):
    sections = []
    num_pages = int(math.ceil(num_sections / SECTIONS_PER_PAGE))
    for page_idx in range(num_pages):
        result = get_sections(course_id, semester, page_idx + 1)
        sections += result
    return sections

def get_sections(course_id, semester, page):
    params = params={
        "course_id": course_id, 
        "semester": semester,
        "page": page, 
        "per_page": SECTIONS_PER_PAGE
    }
    return requests.get("https://api.umd.io/v1/courses/sections", params=params).json()

# call the planet terp api to get the information about a course, including its reviews
def get_course_info(course_id):
    return requests.get("https://api.planetterp.com/v1/course", params={"name": course_id, "reviews": "true"}).json()

# call the planet terp api to get information about a course's grades
def get_course_grades(course_id):
    return requests.get("https://api.planetterp.com/v1/grades", params={"course": course_id}).json()

def parse_semester(semester_raw):
    semester_year = semester_raw[0:4]
    semester_id = int(semester_raw[4:])
    semester_enum = "FALL" if semester_id == 1 else "SPRING" if semester_id == 8 else None
    return semester_year, semester_enum
    
# setup the database. we wipe the database since we want to regenerate all of the information for now.
with db_setup(wipe=True) as conn:
    courses = get_cmsc_courses()
    for course in courses:
        course_id = course["course_id"]
        semester = course["semester"]
        sections = get_all_sections(course_id, semester, len(course["sections"]))
        db_write_section_seats(conn, sections)
        print(f"wrote section info for course {course_id}, semester {semester}")
    
    course_ids = list(map(lambda x: x["course_id"], courses))
    for course_id in course_ids:
        db_write_course_info(conn, get_course_info(course_id))
        db_write_course_grades(conn, course_id, get_course_grades(course_id))
        print(f"processed {course_id}")

wrote section info for course CMSC100, semester 202208
wrote section info for course CMSC106, semester 202208
wrote section info for course CMSC122, semester 202208
wrote section info for course CMSC125, semester 202208
wrote section info for course CMSC131, semester 202208
wrote section info for course CMSC132, semester 202208
wrote section info for course CMSC133, semester 202208
wrote section info for course CMSC216, semester 202208
wrote section info for course CMSC250, semester 202208
wrote section info for course CMSC298A, semester 202208
wrote section info for course CMSC320, semester 202208
wrote section info for course CMSC330, semester 202208
wrote section info for course CMSC335, semester 202208
wrote section info for course CMSC351, semester 202208
wrote section info for course CMSC351H, semester 202208
wrote section info for course CMSC396H, semester 202208
wrote section info for course CMSC411, semester 202208
wrote section info for course CMSC412, semester 202208
wrote s

ZeroDivisionError: float division by zero

# Basic Analysis

In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("project.db")
query = """
SELECT
    courses.id AS course_id,
    courses.dept,
    courses.number,
    courses.credits,
    course_grades.prof_name,
    course_grades.year,
    course_grades.semester,
    course_grades.gpa,
    course_grades.num_drops,
    professor_reviews.rating AS prof_overall_rating
FROM courses
LEFT JOIN course_grades 
    ON course_grades.course = courses.id
LEFT JOIN professor_reviews 
    ON professor_reviews.course = courses.id
    AND professor_reviews.prof_name = course_grades.prof_name
GROUP BY 
    courses.id, 
    course_grades.prof_name, 
    course_grades.semester
"""
df = pd.read_sql(sql=query, con=conn, index_col="course_id")
display(df)

,dept,number,credits,prof_name,year,semester,gpa,num_drops,prof_overall_rating
course_id,,,,,,,,,
CMSC100,CMSC,100,1,Alyssa Neuner,2017.0,SPRING,3.758824,1.0,NaN
CMSC100,CMSC,100,1,Amy Vaillancourt,2015.0,SPRING,3.382609,1.0,NaN
CMSC100,CMSC,100,1,Andrew Nolan,2012.0,SPRING,3.826087,0.0,NaN
CMSC100,CMSC,100,1,Charles Kassir,2014.0,SPRING,3.669444,1.0,NaN
CMSC100,CMSC,100,1,Corie Brown,2021.0,SPRING,3.805882,0.0,5.0
...,...,...,...,...,...,...,...,...,...
CMSC430,CMSC,430,3,Jose Calderon,2020.0,FALL,3.671642,2.0,5.0
CMSC430,CMSC,430,3,Jose Calderon,2020.0,SPRING,3.155814,4.0,5.0
CMSC430,CMSC,430,3,Leonidas Lampropoulos,2021.0,FALL,3.604854,6.0,NaN
